# Tools to build a CGA-based raytracer

Basic Imports

In [1]:
import numpy as np
from clifford.tools.g3c import *

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/Users/manos/opt/anaconda3/envs/elements/lib/python3.8/site-packages/pyganja/__init__.py:2: UserWarning: Failed to import cef_gui, cef functions will be unavailable
  from .script_api import *


Creating a sphere in CGA, located at x with radius r. All objects in CGA have a dual equivalent (the dual of a dual is the object itself).

In [2]:
x = [1,2,3]
r = 10
center = x[0]*e1+x[1]*e2+x[2]*e3
print('center = ', center, '\nradius = ', r)
s = up(center)-1/2*r**2*einf
print('The sphere s in CGA = \n', s)
s_dual = fast_dual(s)

center =  (1^e1) + (2^e2) + (3^e3) 
radius =  10
The sphere s in CGA = 
 (1.0^e1) + (2.0^e2) + (3.0^e3) - (43.5^e4) - (42.5^e5)


Creating a line in CGA, going through points euc_p1 and euc_p2. All objects in CGA have a dual equivalent (the dual of a dual is the object itself).

In [3]:
euc_p1 = [-1,-1,-1]
euc_p2 = [1,1,3]

p1 = up(euc_p1[0]*e1+euc_p1[1]*e2+euc_p1[2]*e3)
p2 = up(euc_p2[0]*e1+euc_p2[1]*e2+euc_p2[2]*e3)

L_dual = (p1)^(p2)^einf #line that goes through (3,4,0),(3,5,2) (and infinity)
L = fast_dual(L_dual)

print('\nThe line L that goes through this point = \n', L)


The line L that goes through this point = 
 (4.0^e12) - (2.0^e13) - (2.0^e14) - (2.0^e15) + (2.0^e23) + (2.0^e24) + (2.0^e25)


The intersection of the sphere s and the line L is simply L^S (^ is denoted as meet)

In [4]:
I = meet(L_dual,s_dual)
print('\nThe Their intersection is : \n', I)


The Their intersection is : 
 (2.0^e13) + (91.0^e14) + (93.0^e15) + (2.0^e23) + (91.0^e24) + (93.0^e25) + (164.0^e34) + (168.0^e35) + (18.0^e45)


What exactly is this I? Let's us the following built in function:

In [5]:
print(interpret_multivector_as_object(I))
#     -1 -> not a blade
#     0 -> a 1 vector but not a point
#     1 -> a euclidean point
#     2 -> a conformal point
#     3 -> a point pair
#     4 -> a circle
#     5 -> a line
#     6 -> a sphere
#     7 -> a plane

3


This means that I is a point pair, i.e., two points bound in a single entity I. To extract these points from I, we can use the following handy function:

In [6]:
A,B = point_pair_to_end_points(I) 
print(A)
print(B)

(5.2321^e1) + (5.2321^e2) + (11.46421^e3) + (92.58893^e4) + (93.58893^e5)
-(2.89877^e1) - (2.89877^e2) - (4.79754^e3) + (19.41107^e4) + (20.41107^e5)


To understand exactly which euclidean points A, B correspond two we use the `down` function:

Check if this corresponds to two points:

In [7]:
print(down(A))
print(down(B))

(5.2321^e1) + (5.2321^e2) + (11.46421^e3)
-(2.89877^e1) - (2.89877^e2) - (4.79754^e3)


The three numbers appearing correspond to the euclidean coordinates. To easily obtain them we can use: 

In [8]:
euc_A = down(A)[e1], down(A)[e2], down(A)[e3]
euc_B = down(B)[e1], down(B)[e2], down(B)[e3]
print(euc_A)
print(euc_B)

(5.232103639216834, 5.232103639216834, 11.464207278433667)
(-2.8987703058834877, -2.8987703058834877, -4.797540611766994)


Built-in functions such as `sphere_line_intersect` and `euc_dist` can help us check is a sphere/line intersect and the distance of two points in CGA, without having to translate back to Euclidean coordinates:

In [9]:
if sphere_line_intersect(L_dual,s_dual): # i.e., if meet(s,L) >0
    print("Line and sphere intersect")
    I = meet(L_dual,s_dual)
    A,B = point_pair_to_end_points(I) 

Line and sphere intersect


In [10]:
dist_from_A = euc_dist(p1, A)
print("The distance of point p1 from point A is ", dist_from_A)


The distance of point p1 from point A is  15.265473940223346


# Visualizing the above

In [11]:
from pyganja import *  

# fixing issue with ganja printing
A = up(down(A))
B = up(down(B))

In [12]:
gs3 = GanjaScene()
gs3.add_objects([up(center)],color=Color.RED)
gs3.add_objects([A],color=Color.YELLOW)
gs3.add_objects([B],color=Color.CYAN)
gs3.add_objects([p1],color=Color.GREEN)
gs3.add_objects([p2],color=Color.MAGENTA)
gs3.add_objects([L_dual],color=Color.MAGENTA) # we have to draw the dual of the line to visualize it
gs3.add_objects([s],color=Color.BLUE)
draw(gs3,scale=0.25)

<IPython.core.display.Javascript object>